In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np

### Importing the nutrients dataframe and a general dataframe to get the 'nutriscore_grade' column

In [17]:
df = pd.read_csv('Data/cleaned_csv/openfoodfacts_nutrients_clean.csv', sep=',', header=0)
df_grade = pd.read_csv('Data/cleaned_csv/openfoodfacts_france_only.csv', sep=',', header=0)

C:\Users\tiger\AppData\Local\Temp\ipykernel_8388\4021274885.py:2: DtypeWarning: Columns (0,23,26,36,71,74,75,78,79,81,82,83,84,85,86,87,88,89,91,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df_grade = pd.read_csv('Data/cleaned_csv/openfoodfacts_france_only.csv', sep=',', header=0)


### Encoding 'pnns_groups_1' with OneHotEncoder

In [18]:
encoder = OneHotEncoder(sparse_output=False)

pnns_encoded = encoder.fit_transform(df[['pnns_groups_1']])

pnns_encoded_df = pd.DataFrame(pnns_encoded, columns=encoder.get_feature_names_out(['pnns_groups_1']))

df = pd.concat([df, pnns_encoded_df], axis=1).drop(columns=['pnns_groups_1'])

KeyError: "None of [Index(['pnns_groups_1'], dtype='object')] are in the [columns]"

### Ordinal encoding of 'nutriscore_grade'

In [19]:
df_grade['nutriscore_grade'].replace({
    'a': 1,
    'b': 2,
    'c': 3,
    'd': 4,
    'e': 5
}, inplace=True)

C:\Users\tiger\AppData\Local\Temp\ipykernel_8388\883079500.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_grade['nutriscore_grade'].replace({
C:\Users\tiger\AppData\Local\Temp\ipykernel_8388\883079500.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_grade['nutriscore_grade'].replace({


### Data normalization

In [20]:
scaler = MinMaxScaler()

df[[col for col in df.columns if col.endswith('_100g')]] = scaler.fit_transform(df[[col for col in df.columns if col.endswith('_100g')]])

### Splitting data for training and testing

In [21]:
X = df
y = df_grade['nutriscore_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

### Training with LogisticRegression

In [22]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy :", accuracy)
print("Confusion matrix :\n", conf_matrix)
print("Classification report :\n", report)

Accuracy : 0.57489342328546
Confusion matrix :
 [[ 5460  1021  1074    28     3]
 [ 2069  1449  2389   276     6]
 [ 1453   739  4512  3144   402]
 [  297   383  1201 11504  1391]
 [   86   146   975  3459  4855]]
Classification report :
               precision    recall  f1-score   support

           1       0.58      0.72      0.64      7586
           2       0.39      0.23      0.29      6189
           3       0.44      0.44      0.44     10250
           4       0.62      0.78      0.69     14776
           5       0.73      0.51      0.60      9521

    accuracy                           0.57     48322
   macro avg       0.55      0.54      0.53     48322
weighted avg       0.57      0.57      0.56     48322



c:\Users\tiger\OneDrive\Documents\GitHub\nutriscore_prediction_app\env\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Training with RandomForestClassifier

In [23]:
model = RandomForestClassifier(
    n_estimators=100,       # More trees = stronger model
    max_depth=10,           # Reducing max_depth = reducing overfitting
    max_features='sqrt',
    min_samples_split=8,
    min_samples_leaf=2,
    bootstrap=True,
    random_state=42
    )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy :", accuracy)
print("Confusion matrix :\n", conf_matrix)
print("Classification report :\n", report)

Accuracy : 0.8445221638177228
Confusion matrix :
 [[ 6492   768   239    15    72]
 [  937  4128   964    63    97]
 [  136   645  8091  1162   216]
 [  136    40   551 13625   424]
 [   74   133    88   753  8473]]
Classification report :
               precision    recall  f1-score   support

           1       0.83      0.86      0.85      7586
           2       0.72      0.67      0.69      6189
           3       0.81      0.79      0.80     10250
           4       0.87      0.92      0.90     14776
           5       0.91      0.89      0.90      9521

    accuracy                           0.84     48322
   macro avg       0.83      0.82      0.83     48322
weighted avg       0.84      0.84      0.84     48322



### Cross-Validation on RandomForestClassifier with 10 folds

In [24]:
k = 10
scores = cross_val_score(model, X, y, cv=k)

print("Cross-validation scores for each fold:", scores)
print("Average cross-validation score:", np.mean(scores))

Cross-validation scores for each fold: [0.83104144 0.84761757 0.86208288 0.85761291 0.85050442 0.8494133
 0.83221581 0.82017135 0.83618923 0.80822003]
Average cross-validation score: 0.8395068949549046


### GridSearch to determine best parameters for the RandomForestClassifier

In [80]:
parameters = {'min_samples_leaf': [2, 4, 6, 8, 10]}

model = RandomForestClassifier(n_estimators=100, max_depth=10, max_features='sqrt', min_samples_split=8, bootstrap=True, random_state=42)

gs = GridSearchCV(model, parameters, cv=5, scoring='roc_auc_ovr', verbose=1)
gs.fit(X_train, y_train)
display(gs.best_params_)
display(gs.best_score_)
display(gs.best_estimator_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


{'min_samples_leaf': 2}

np.float64(0.9682570856775389)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=8,
                       random_state=42)

### Saving the trained model as a save file

In [25]:
from joblib import dump

In [26]:
dump(model, 'nutriscore_model.joblib')

['nutriscore_model.joblib']